# Connecting to Postgres Lab

### Introduction

In this lesson, we'll setup an RDS instance and use it to serve as the database for our foursquare flask api.  Let's get started.

### Setting it up

* We should begin by going to the AWS console, finding the RDS service, and creating a new database.  
* Then select the postgres database, and the free tier.
* From here, it's time to add in the settings for the database.  Choose an instance identifier of `foursquare-flask-api`.  And set a master username and password.  

> **Note**: Make sure to write down these values as we'll need them to connect to our database later.

* Then, under connectivity make sure that there is public access to the database.  

> This will allow us to access the database with just the ip address, username, and password.


* After checking that the configuration is correct, create the database.

### Checking our Work

Go to the dashboard and click on the foursquare-flask-api.  While it is still just creating, we will be able to see the settings of the database instance.

<img src="./foursquare_flask_api.png" width="100%">

If we click on it, we'll see the following.

<img src="./foursquare_api.png" width="100%">

* **Make sure** you also check the security group associated with the database.  Confirm that there is public access on port 5432.  Please do that now.

### Connecting to our instance

Now let's connect to our instance and create a database called `foursquare_production`.

Once connected, listing the databases and we should see the following.

<img src='./listing_databases.png' width="80%">

From there, we can run migrations to create the production tables.  

> We can use the same `psql` command we normally do to run migrations, just this time specify a `--host` parameter along with the file, and the database with `-d`.  The host parameter will be the public ip address for the database.

If you run the following from the shell, filling in the value for the host, you can check that the tables were created.

`psql -U postgres -d foursquare_production --host=specify_host -c "\dt"`

<img src="./view_relations.png" width="60%">

### Connecting our Application

Ok, now let's connect our flask application to our production database.  We'll do this in two steps.  First, we'll first make the change locally and check that it's working.  And then we'll apply the change to our Flask API running on the EC2 instance.

1. Making the Change Locally

To connect our local flask application to the production database.  We need to change our environmental variables.  Remember that our environmental variables are stored in the `.env` file.  We can change those values locally, and run our application locally to that we are connected to the RDS instance.

> Run the application using a port of `0.0.0.0`.

> If you visit the venues resource, we should see the following.  

> **Note** Instead of a url of `0.0.0.0/venues`, you can also visit `http://127.0.0.1:5000/venues`.

<img src="./venues.png" width="80%">

This is great, as we are now connected to our database on AWS.

* A closer examination

Before moving on, it's good to make sure you understand how this works.

If you look at the db.py file, you will see the following.

```python
from settings import DB_HOST, DB_NAME, DB_PASSWORD, DB_USER, DEBUG, TESTING

conn = psycopg2.connect(host = DB_HOST, database = DB_NAME,
        user = DB_USER, password = DB_PASSWORD)

def get_db():
    if "db" not in g:
        g.db = psycopg2.connect(user = current_app.config['DB_USER'],
                password = current_app.config['DB_PASSWORD'],
                host = DB_HOST,
            dbname = current_app.config['DATABASE'])
    return g.db
```

2. Connecting on AWS

A. Get the Flask Code Running


* We will have to change the `run.py` file so that we are running on the correct host and port.  


B. Setup the EC2 machine

* Next create a new EC2 instance, and make sure the security group is setup properly.

* Then `scp` the code over to the running EC2 instance, or create a new EC2 instance if one is not available.

* Then ssh into the ec2 machine and install the packages in the `requirements.txt`

* If you did not already, create a tmux session.

`tmux attach-session -t flask_lab`

* Connect to the instance, create a new tmux session, and boot up the flask app with a call to `sudo python3 run.py`.  Detach from the tmux session, and confirm that we our application works by making a request (via HTTP not HTTPS in the browser).
    
**Note**: If you **cannot connect**, check the security group associated with the EC2 machine and make sure that port 80 is publicly available.

Our flask application should be able to connect to the database without breaking (there just won't be any data in there).

<img src="./venues_from_ec2.png" width="80%">

### Adding some data

Let's pull down some venues into our production database by using our application code to connect to the foursquare API and filling our database with venue records.  

> We can do this by again sshing into our EC2 instance and this time running:

> `sudo python3 manage.py build_venues "40.7,-74" "tacos"`

This uses the command defined in the manage.py file.  Refreshing the venues url from our api, we should see the following.

<img src="./production_venues_filled.png" width="80%">

### Summary

In this lesson, we connected our application to our production database.  The key steps were to create our RDS instance while making our RDS instance publicly accessible.  Then we need to configure our RDS instance with the master username, password.  This along with the RDS ip address, allowed us to connect to our instance create our production database and run our migration files.

We then change our username, password, and host address in our `.env` file to our flask application reference the correct database in production.  We updated the code on our EC2 instance, and re-ran our flask application with in tmux.  Then after getting this working, we used our cli command to populate our database.  